# Ensemble of all models

In [239]:
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
import numpy as np

In [18]:
res1 = pd.read_csv("C:\\Users\\elonm\\Desktop\\res_lda.csv", header=None)
res2 = pd.read_csv("C:\\Users\\elonm\\Desktop\\res_nn.csv", header=None)
res3 = pd.read_csv("C:\\Users\\elonm\\Desktop\\res_XGB.csv")
res4 = pd.read_csv("C:\\Users\\elonm\\Desktop\\res_SVM.csv")
# res5 = pd.read_csv("C:\\Users\\elonm\\Desktop\\res_lda.csv", header=None)

In [88]:
def bin(a):
    if a == 1:
        return 0
    else:
        return 1
res1[0] = res1[0].apply(lambda x: bin(x))

In [204]:
y = pd.read_csv(r"E:\Yelp\Unfiltered Data\WorkingSet\y_test", header=None) 
y[1] = y[1].apply(lambda x: bin(x))
print(y.shape)

(121692, 2)


In [92]:
index = []
for i in range(actual.shape[0]):
    index.append(i)

X = pd.DataFrame({'a':res1[0], 'b':res2[0], 'c':res3['0'], 'd':res4['0']}, index=index)

## Using a small neural network to derive the weights of all results

In [249]:
model = Sequential()
model.add(Dense(4, activation='sigmoid', input_dim=4))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 4)                 20        
_________________________________________________________________
dropout_5 (Dropout)          (None, 4)                 0         
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________


In [250]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y[1], test_size=0.33, random_state=42)

In [251]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
81533/81533 [==============================] - 2s 30us/step - loss: 0.4038 - acc: 0.8679
Epoch 2/5
81533/81533 [==============================] - 2s 23us/step - loss: 0.3931 - acc: 0.8679
Epoch 3/5
81533/81533 [==============================] - 2s 23us/step - loss: 0.3896 - acc: 0.8679
Epoch 4/5
81533/81533 [==============================] - 2s 23us/step - loss: 0.3881 - acc: 0.8679
Epoch 5/5
81533/81533 [==============================] - 2s 23us/step - loss: 0.3875 - acc: 0.8679


In [252]:
scores = model.evaluate(X_train, y_train)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

81533/81533 [==============================] - 1s 14us/step

acc: 86.79%


In [253]:
aa = []
for i in y_pred:
    if i <= 0.15:
        aa.append(1)
    else:
        aa.append(0)

In [254]:
# from sklearn.metrics import classification_report
# print(classification_report(y_pred, y_test))
y_pred = np.squeeze(model.predict(X_test))
threshold = 0.2
print(classification_report(y_test, y_pred > threshold))

             precision    recall  f1-score   support

          0       0.88      0.98      0.92     34892
          1       0.37      0.09      0.15      5267

avg / total       0.81      0.86      0.82     40159



## Ensembling results using Max Votes

In [233]:
y_self = []

for i in range(y.shape[0]):
    sum = 0
    if X['a'][i] == 1:
        sum = sum + 1
    if X['b'][i] == 1:
        sum = sum + 1
    if X['c'][i] == 1:
        sum = sum + 1
    if sum >= 2:
        y_self.append(1)
    else:
        y_self.append(0)

In [234]:
print(classification_report(y[1], y_self))

             precision    recall  f1-score   support

          0       0.86      0.46      0.60    105656
          1       0.13      0.52      0.21     16036

avg / total       0.77      0.47      0.55    121692



# Results:
## Accuracy: 86.79%
## Precision: 81%
## f1 Score: 0.81
## Recall: 0.86